In [17]:
import re
import nltk
import praw
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
import psycopg2
#nltk.download('stopwords', quiet=True)

In [35]:
db_connection = psycopg2.connect(
            user='postgres', password='Data2023',
            host='mads-capstone.cmohac77hep9.eu-north-1.rds.amazonaws.com', port=5432,
            database="mads"
        )

db_connection.autocommit = True

cursor = db_connection.cursor()

In [33]:
def PRAW_for_dashboard():
    def preprocess_df(df):
        def preprocess_text(text):

            # Remove specific unwanted characters
            text = re.sub(r'[^A-Za-z0-9\s,.!?;:()\'\"-]', '', text)

            # Strip whitespace
            text = text.strip()

            return text

        try:
            df['title'] = df['title'].fillna('')
            df['title'] = df['title'].apply(preprocess_text)

            # Create a new column with shifted values
            df['title_prior'] = df['title'].shift(1)
            # Drop rows where the value in 'ColumnToCheck' is the same as in 'ShiftedColumn'
            df = df[df['title'] != df['title_prior']]
            # drop the 'ShiftedColumn'
            df = df.drop('title_prior', axis=1)

        except:
            print(f'No title found, skipping')


        # handle blank 
        df['selftext'] = df['selftext'].fillna('') 

        # preprocess selftext
        df['selftext'] = df['selftext'].apply(preprocess_text)

        # localize the UTC time stamp
        df['created_EST_date'] = pd.to_datetime(df['created_utc'], unit='s').dt.tz_localize('UTC').dt.tz_convert('America/New_York').dt.date


        return df
    
    def find_tickers(df):
 
        # Load tickers from a CSV file
        stocks = pd.read_csv(r'/root/Git Repo/SIADS_Capstone_Group17/Data/nasdaq_screener.csv')

        # Directly convert the 'Symbol' column to a set
        tickers_set = set(stocks['Symbol'])

        # Use set union to add additional tickers
        additional_tickers = {'BBBY'}
        tickers_set = tickers_set.union(additional_tickers)

        # Adjust the pattern to optionally include a leading '$'
        # pattern = r'\b\$?(?:' + '|'.join(tickers_as_strings) + r')\b'
        # pattern = r'\b\$?(?:\(?)(?:' + '|'.join(tickers_as_strings) + r')(?:\)?)\b'
        pattern = r'\b[A-Z]{2,5}\b'
        compiled_pattern = re.compile(pattern)



        blacklist = {
    #                 {'I', 'ELON', 'WSB', 'THE', 'A', 'ROPE', 'YOLO', 'TOS', 'CEO', 'DD', 'IT', 'OPEN', 'ATH', 'PM', 'IRS', 'FOR',
    #              'DEC', 'BE',
                    'IMO',# 'ALL', 'RH', 'EV', 'TOS', 'CFO', 'CTO',
                    'DD',
                    #'BTFD', 'WSB', 'OK', 'PDT', 'RH', 'KYS', 'FD',
    #              'TYS', 
                    'US',
                    'USA',
                    # 'IT', 'ATH', 'RIP', 'BMW', 'GDP', 'OTM', 'ATM', 'ITM', 
                    'IMO', 'LOL', 'AM', 'BE', 'PR', 'PRAY',
    #              'PT', 'FBI', 'SEC', 'GOD', 'NOT', 'POS', 'FOMO', 'TL;DR',
                    'EDIT', 'STILL', 'WTF', 'RAW', 'PM', 'LMAO', 'LMFAO',
    #              'ROFL', 'EZ', 'RED', 'BEZOS', 'TICK', 'IS', 'PM', 'LPT', 'GOAT', 'FL', 'CA', 'IL', 'MACD', 'HQ', 'OP', 'PS', 'AH',
    #              'TL', 'JAN', 'FEB', 'JUL', 'AUG', 'SEP', 'SEPT', 'OCT', 'NOV', 'FDA', 'IV', 'ER', 'IPO', 'MILF', 'BUT', 'SSN', 'FIFA',
    #              'USD', 'CPU', 'AT', 'GG', 'Mar', 

    #                # Jake added
                    'RUN', # common
                    'SAY', # common
                    'EOD', # end of day
                    'BIG', # common
                    'LOW', # low / high
                    'RSI', #relative strenght
                    'DT', #double top
                    'HUGE',
                    'U', # you
                    'AI', # Artificial Intelligence
                    'DC', # washington DC
                    'J', # as in J Powell
                    'ES', # E-mini SP future
                    'F', # f*ck
                    'GO',
                    'UK', # United Kingdom
                    'EU', # european union
                    'RH', # Robinhood, not Restoration Hardware
                    'E', # E*trade brokerage
                    'L', # L for loss, P&L etc
                    'R', # common 
                    'K', # OK
                    'B', # common in BBBY odd spacing (spam?)
                    'TD', # TD Ameritrade brokerage
                    'RYAN', # Ryan Cohen, CEO of GME
                    'NYC', # New York City
                    'REG', # reg SHO 
                    'SHO', # reg SHO 
                    'NEXT', # common
                    'FREE', # spam
                    'DM', # direct message
                    'TV', # television
                    'ENS', # ethereum name service, spam
                    'IRS', # internal revenue service
                    'PR', # public relations
                    'IQ', # intelligence quotient
                    'VS', # versus
                    'PT', # price target
                    'IBKR', # interactive brokers
                    'GOOD', # common
                    'OPEN', # market open
                    'FCF', # free cash flow


                    }

        combined_blacklist = set(blacklist) | set(word.upper() for word in stopwords.words('english'))


        def find_tickers(text, compiled_pattern, tickers_set, blacklist_set):
            # Find all matches
            potential_tickers = compiled_pattern.findall(text)
            # Filter matches against the tickers list and ensure they are not in the blacklist
            return list(set([ticker for ticker in potential_tickers if ticker in tickers_set and ticker not in combined_blacklist]))

        try:
            df['title_tickers'] = df['title'].apply(lambda x: find_tickers(x, compiled_pattern, tickers_set, combined_blacklist))
        except KeyError:
            print('title not found, working with comments?')


        df['selftext_tickers'] = df['selftext'].apply(lambda x: find_tickers(x, compiled_pattern, tickers_set, combined_blacklist))

        df['tickers'] = [list(set(x + y)) for x, y in zip(df['title_tickers'], df['selftext_tickers'])]

        return df
    
    def add_vader_sentiment(df):

        vader = SentimentIntensityAnalyzer()

        added_words = {
                'citron': -4.0,  
                'hidenburg': -4.0,        
                'moon': 4.0,
                'highs': 2.0,
                'mooning': 4.0,
                'long': 2.0,
                'short': -2.0,
                'call': 4.0,
                'calls': 4.0,    
                'put': -4.0,
                'puts': -4.0,    
                'break': 2.0,
                'tendie': 2.0,
                'tendies': 2.0,
                'town': 2.0,     
                'overvalued': -3.0,
                'undervalued': 3.0,
                'buy': 4.0,
                'sell': -4.0,
                'gone': -1.0,
                'gtfo': -1.7,
                'paper': -1.7,
                'bullish': 3.7,
                'bearish': -3.7,
                'bagholder': -1.7,
                'stonk': 1.9,
                'green': 1.9,
                'money': 1.2,
                'print': 2.2,
                'rocket': 2.2,
                'bull': 2.9,
                'bear': -2.9,
                'pumping': -1.0,
                'sus': -3.0,
                'offering': -2.3,
                'rip': -4.0,
                'downgrade': -3.0,
                'upgrade': 3.0,     
                'maintain': 1.0,          
                'pump': 1.9,
                'hot': 1.5,
                'drop': -2.5,
                'rebound': 1.5,  
                'crack': 2.5,
                '🚀': 3, # Jake ADDED THESE
                '🌕': 3, # Jake ADDED THESE
                'YOLO': 4, # Jake ADDED THESE
                'ripping': 3,# Jake ADDED THESE
                'regarded': 0, # Jake ADDED THESE
                'squeeze':3, # Jake ADDED THESE
                }

        vader.lexicon.update(added_words)

        def safe_sentiment(text):
            try:
                # Ensure the input is a non-empty string
                if not isinstance(text, str) or not text.strip():
                    return 0

                # Analyze the sentiment
                sentiment_dict = vader.polarity_scores(text)
                return sentiment_dict.get('compound', 0)
            except Exception as e:
                print(f"Error processing text: '{text}' (type: {type(text)}). Error: {e}")
                return 0

        # Apply the function
        try:
            df['title_sentiment'] = df['title'].apply(safe_sentiment)
        except:
            print('Titles not found, is this a comments file?')
            df['title_sentiment'] = 0

        df['selftext_sentiment'] = df['selftext'].apply(safe_sentiment)


        def non_zero_average(row):
            sentiments = [row['title_sentiment'], row['selftext_sentiment']]
            non_zero_sentiments = [s for s in sentiments if s != 0]

            if not non_zero_sentiments:
                return 0  # Return 0 if both sentiments are zero

            return sum(non_zero_sentiments) / len(non_zero_sentiments)

        # Apply the function to calculate overall sentiment
        df['overall_sentiment'] = df.apply(non_zero_average, axis=1)

        df['score_weighted_sentiment'] = df['overall_sentiment'] * df['score']

        return df
    
    def get_reddit_praw_submissions(limit):
    
        client_id = 'aWEYVIaAoJGlCPja3awh0A'
        secret = 'gOR5FfkvsTH3MJ0IHRSImToTwt0PSQ'

        reddit = praw.Reddit(
                            client_id=client_id,
                            client_secret=secret,
                            user_agent="MADS/0.1 by TeamSafari",
                        )
        submissions_data = []

        for submission in reddit.subreddit("wallstreetbets").new(limit=limit):
            # print(dir(submission))
            data = {
                'id': submission.id,
                'subreddit_id': submission.subreddit_id,
                'subreddit': submission.subreddit,
                'author': submission.author,
                'created_utc': submission.created_utc,
                'permalink': submission.permalink,
                'title': submission.title,
                'selftext': submission.selftext,
                'num_comments': submission.num_comments,
                'score': submission.score,
                'flair': submission.link_flair_text,
                'removal_reason':submission.removal_reason,


                # Add more fields as needed
            }
            submissions_data.append(data)

        df = pd.DataFrame(submissions_data)
        return df
    
    
    df = get_reddit_praw_submissions(limit=1000)
    praw_df = preprocess_df(df)
    praw_df = find_tickers(praw_df)
    praw_df = add_vader_sentiment(praw_df)
    
    
    # display(praw_df)
    
    
    # create cumulative_sentiment_sorted_df
    exploded_df = praw_df.explode('tickers')
    cumulative_sentiment = exploded_df.groupby('tickers')['overall_sentiment'].sum().reset_index() # Group by 'tickers'
    cumulative_sentiment.columns = ['Ticker', 'Cumulative Overall Sentiment'] # Rename columns for clarity
    cumulative_sentiment_sorted = cumulative_sentiment.sort_values(by='Cumulative Overall Sentiment', ascending=False)
    # display(cumulative_sentiment_sorted)

    # Group by 'tickers' and sum the 'score_weighted_sentiment'
    cumulative_weighted_sentiment = exploded_df.groupby('tickers')['score_weighted_sentiment'].sum().reset_index()
    cumulative_weighted_sentiment.columns = ['Ticker', 'Cumulative Weighted Sentiment'] # Rename columns for clarity
    cumulative_weighted_sentiment_sorted = cumulative_weighted_sentiment.sort_values(by='Cumulative Weighted Sentiment', ascending=False)
    cumulative_weighted_sentiment_sorted['Date'] = praw_df['created_EST_date']
    cumulative_weighted_sentiment_sorted['Date'].fillna(praw_df['created_EST_date'].unique()[0],inplace=True)
    # display(cumulative_weighted_sentiment_sorted)

    daily_sentiment_df = cumulative_sentiment_sorted.merge(cumulative_weighted_sentiment_sorted, on='Ticker')
    
    # Explode the DataFrame if 'tickers' column contains lists
    # exploded_df = praw_df.explode('tickers')

    expected_columns = ['tickers', 'title', 'permalink', 'overall_sentiment']

    # Check if all expected columns are in the DataFrame
    if not all(column in exploded_df.columns for column in expected_columns):
        raise ValueError("DataFrame does not contain the expected columns.")

    # Base URL to prepend to permalinks
    base_url = 'https://www.reddit.com/'
    
    # Aggregate submission details with the base URL prepended to permalinks
    #submission_details = exploded_df.groupby('tickers').apply(
    #    lambda x: [(title, base_url + permalink, sentiment) 
    #               for title, permalink, sentiment in zip(x['title'], x['permalink'], x['overall_sentiment'])]
    #).reset_index(name='Submissions')

    # Rename columns for clarity
    #submission_details.columns = ['Ticker', 'Submissions']
    exploded_df = exploded_df.rename(columns={"tickers": "Ticker"})
    exploded_df = exploded_df[["Ticker", "title", "permalink", "overall_sentiment"]]

    # Merge with daily_sentiment_df
    daily_sentiment_df = daily_sentiment_df.merge(exploded_df, on='Ticker', how='left')
    daily_sentiment_df["title"] = daily_sentiment_df["title"].str.replace(",", ";")
    
    return daily_sentiment_df

In [34]:
%%time

final_df = PRAW_for_dashboard()

#final_df

CPU times: user 1.88 s, sys: 17.9 ms, total: 1.9 s
Wall time: 14.7 s


,Ticker,Cumulative Overall Sentiment,Cumulative Weighted Sentiment,Date,title,permalink,overall_sentiment
0,SAVE,8.89415,310.8972,2023-12-07,Odds of the SAVE case being decided on or befo...,/r/wallstreetbets/comments/18epb7p/odds_of_the...,0.78770
1,SAVE,8.89415,310.8972,2023-12-07,SAVE merger question,/r/wallstreetbets/comments/18cl5jf/save_merger...,0.77200
2,SAVE,8.89415,310.8972,2023-12-07,SAVE trial bullish quotes from the judge,/r/wallstreetbets/comments/18by8rd/save_trial_...,0.89165
3,SAVE,8.89415,310.8972,2023-12-07,What happens to SAVE if JetBlue merger is bloc...,/r/wallstreetbets/comments/18bjkra/what_happen...,0.64795
4,SAVE,8.89415,310.8972,2023-12-07,What are your thoughts on the SpiritJetblue me...,/r/wallstreetbets/comments/18aq3j1/what_are_yo...,0.76030
...,...,...,...,...,...,...,...
414,BBVA,-0.95110,-6.6577,2023-12-09,Shorting select Argentinian companies,/r/wallstreetbets/comments/185f4oj/shorting_se...,-0.95110
415,NOV,-0.95110,-6.6577,2023-12-07,Shorting select Argentinian companies,/r/wallstreetbets/comments/185f4oj/shorting_se...,-0.95110
416,YPF,-0.95110,-6.6577,2023-12-06,Shorting select Argentinian companies,/r/wallstreetbets/comments/185f4oj/shorting_se...,-0.95110
417,DHI,-1.30180,-45.7194,2023-12-08,Did Warren Buffett make a mistake? BUY DHI PUTS.,/r/wallstreetbets/comments/18cz9ap/did_warren_...,-0.59340


query = '''ALTER TABLE reddit_sentiment_by_date
                ADD COLUMN title text,
                ADD COLUMN permalink text,
                ADD COLUMN overall_sentiment float;
'''


cursor.execute(query)

In [36]:
final_df.to_csv("latest_sentiment.csv", index=False, header=False)

In [41]:
#!tail latest_sentiment.csv

In [39]:
with open("latest_sentiment.csv", "rb") as fp:
    cursor.copy_from(fp, "reddit_sentiment_by_date", sep=",", null="", columns=["ticker", "cumm_overall_sentiment", "cumm_weighted_sentiment", "record_date",  "title", "permalink", "overall_sentiment"])

In [40]:
select_query = "SELECT * FROM reddit_sentiment_by_date ORDER BY record_date DESC LIMIT 100;"

cursor.execute(select_query)

rows = cursor.fetchall()

records = []
for row in rows:
    records.append([row[0], row[1],row[2], row[3], row[4], row[5], row[6], row[7]])
    
df = pd.DataFrame(records, columns=["id", "ticker", "cumm_overall_sentiment", "cumm_weighted_sentiment", "record_date", "title", "permalink", "overall_sentiment"])

print(df.shape)

df.head()

(100, 8)


,id,ticker,cumm_overall_sentiment,cumm_weighted_sentiment,record_date,title,permalink,overall_sentiment
0,162372,AAOI,-0.27480,-5.4960,2023-12-10,bought AAOI 3 month ago at 14.80; went down as...,/r/wallstreetbets/comments/188r675/bought_aaoi...,-0.27480
1,162333,AAL,0.08140,0.2442,2023-12-10,Long Airline stocks Jets AAL UAL as of today,/r/wallstreetbets/comments/186rw7p/long_airlin...,0.08140
2,162140,AAPL,1.70275,30.5043,2023-12-10,Apple's head of iphone and Apple watch design ...,/r/wallstreetbets/comments/18ecxrr/apples_head...,0.00000
3,162141,AAPL,1.70275,30.5043,2023-12-10,Dont be a Chad! Sometimes we have to seek Valu...,/r/wallstreetbets/comments/18akdap/dont_be_a_c...,0.87495
4,162142,AAPL,1.70275,30.5043,2023-12-10,Paramount and Apple teaming Up?,/r/wallstreetbets/comments/188bdh8/paramount_a...,0.37550


In [42]:
db_connection.close()